In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf

import gensim
import jamo
from PIL import Image
from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm
tqdm.pandas()

from unicode import join_jamos

In [2]:
DATA_TYPE = 'full-nouns-trigram'
DATA_DIR = 'lol'

DIM = 200

if 'nouns' in DATA_TYPE:
    MAXLEN_TITLE = 16
    MAXLEN_LEDE = 24
    MAXLEN_CONTENTS = 128
else:
    MAXLEN_TITLE = 32
    MAXLEN_LEDE = 64
    MAXLEN_CONTENTS = 256

In [3]:
MINING_DIR = '1-mining'
TOKENIZING_DIR = '3-tokenizing'
EMBEDDING_DIR = '4-embedding'
TRAINING_DATA_DIR = '5-training-data'

mining_dir = Path(MINING_DIR) / DATA_DIR
tokenizing_dir = Path(TOKENIZING_DIR) / DATA_DIR
embedding_dir = Path(EMBEDDING_DIR) / DATA_DIR
training_data_dir = Path(TRAINING_DATA_DIR) / DATA_DIR

embedding_dir.mkdir(parents=True, exist_ok=True)
training_data_dir.mkdir(parents=True, exist_ok=True)

In [4]:
wv_model_path = embedding_dir / 'news-{}-wv{}.bin'.format(DATA_TYPE, DIM)
wv_vector_path = embedding_dir / 'news-{}-wv{}.vec'.format(DATA_TYPE, DIM)

ft_model_path = embedding_dir / 'news-{}-ft{}.bin'.format(DATA_TYPE, DIM)
ft_vector_path = embedding_dir / 'news-{}-ft{}.vec'.format(DATA_TYPE, DIM)

### data

In [5]:
df = pd.read_csv(embedding_dir / 'news-{}.csv'.format(DATA_TYPE)).dropna()
df.vod = df.vod.astype('int64').astype('category')
df.thumbnail = df.thumbnail.astype('int64').astype('category')

np.save(training_data_dir / 'news-view.npy', np.asarray(df.view, 'int32'))
df.tail()

,date,date_input,date_modify,ranking,thumbnail,title,lede,office,contents,vod,view
51811,20200430,20200430 17:49,20200430 17:56,26,1,승강_전 다이브 승리 샌드박스 서라벌 세트 선승,일_종각 파크 섬머_스플릿 승강_전 최종_전 세트 샌드박스_게이밍 서라벌_게이밍 승...,인벤,일_종각 파크 섬머_스플릿 승강_전 최종_전 세트 샌드박스_게이밍 서라벌_게이밍 승...,0,1166
51812,20200430,20200430 16:14,20200430 16:31,27,1,승강_전 서라벌 샌드박스 베스트_라인업 맞대결,서머 최종_전 대결 서라벌_게이밍 위 샌드박스_게이밍 사진 라이엇_게임즈 제공 마지...,데일리e스포츠,마지막 승강_전 승리 서라벌_게이밍 샌드박스_게이밍 승강_전 기용 베스트_라인업 출...,0,1057
51813,20200430,20200430 17:50,20200430 17:50,28,1,일방 전투 승리 샌드박스_게이밍 서라벌_게이밍 상대 세트 승리,일 우리은행 승강_전 최종진_출전 샌드박스_게이밍 서라벌_게이밍 세트 경기 초반 기...,엑스포츠뉴스,일 우리은행 승강_전 최종진_출전 샌드박스_게이밍 서라벌_게이밍 세트 경기 초반 기...,0,835
51814,20200430,20200430 18:05,20200430 18:05,29,1,승강_전 샌드박스 카운터 기선_제압,용준 기자 최후 승강_전 샌드박스 다이브 공세 흐름 샌드박스 승강_전 최종_전 서전,OSEN,최후 승강_전 샌드박스 다이브 공세 흐름 샌드박스 승강_전 최종_전 서전 승리 장식...,0,490
51815,20200430,20200430 18:51,20200430 18:51,30,1,승강_전 샌드박스 집중력 혼전 승리 리드,용준 기자 기회 샌드박스 최후 승강_전 생환 승 샌드박스 일_오후_서울,OSEN,기회 샌드박스 최후 승강_전 생환 승 샌드박스 일_오후_서울 종로_롤_파크 아레나 ...,0,460


### word2vec, fasttext embedding

In [6]:
train_range = df.date.apply(lambda x: 20150000 < x < 20200000)
test_range = df.date.apply(lambda x: 20200000 < x < 20210000)

def my_train_test_split(data):
    return data[train_range], data[test_range]

def my_train_val_test_split(data):
    train, test = my_train_test_split(data)
    train, val = train_test_split(train, test_size=0.1, random_state=119)
    return train, val, test

In [7]:
sentences = np.asarray([x.split(' ') + y.split(' ') for x, y in zip(df.title, df.contents)])
sentences, _, _ = my_train_val_test_split(sentences)

In [8]:
wv_mod = gensim.models.Word2Vec(sentences, sg=1, size=DIM)
wv_mod.save(str(wv_model_path))
wv_mod.wv.save_word2vec_format(str(wv_vector_path))

In [9]:
ft_mod = gensim.models.FastText(sentences, sg=1, size=DIM)
ft_mod.save(str(ft_model_path))
ft_mod.wv.save_word2vec_format(str(ft_vector_path))

### preprocessing thumbnail

In [6]:
thumbnails = []
for row in tqdm(df.itertuples(index=False), total=len(df)):
    mining_date_dir = mining_dir / str(row.date)
    thumbnail_glob = '{:04d}-*-thumbnail.jpg'.format(row.ranking)
    try:
        thumbnail_path = next(mining_date_dir.glob(thumbnail_glob))
    except StopIteration:
        thumbnail = np.asarray(Image.new('RGB', (210, 122), (255, 255, 255)))
    else:
        thumbnail = np.asarray(Image.open(thumbnail_path).convert('RGB'))
    thumbnails.append(thumbnail)

thumbnails = np.asarray(thumbnails, 'float32') / 255.0
np.save(training_data_dir / 'news-thumbnail.npy', thumbnails)

### embedding title & lede & contents

In [6]:
import fasttext
import fasttext.util

fasttext.util.download_model('ko', if_exists='ignore')

ft = fasttext.load_model('cc.ko.300.bin')
fasttext.util.reduce_model(ft, 200)

EMBED_NAME = 'ft-pre'
get_vector = ft.get_word_vector

In [21]:
#wv_mod = gensim.models.Word2Vec.load(str(wv_model_path))
#EMBED_NAME = 'wv'

def get_vector(x):
    try:
        return wv_mod.wv.get_vector(x)
    except KeyError:
        return None

In [24]:
ft_mod = gensim.models.FastText.load(str(ft_model_path))
EMBED_NAME = 'ft'
get_vector = ft_mod.wv.get_vector

In [7]:
titles = []
ledes = []
contentss = []
for title, lede, contents in zip(df.title, df.lede, df.contents):
    titles.append(np.asarray([x for x in [get_vector(x) for x in title.split(' ')] if x is not None]))
    ledes.append(np.asarray([x for x in [get_vector(x) for x in lede.split(' ')] if x is not None]))
    contentss.append(np.asarray([x for x in [get_vector(x) for x in contents.split(' ')] if x is not None]))

for x in [[len(x) for x in y] for y in [titles, ledes, contentss]]:
    print(max(x), min(x), np.mean(x), np.std(x))

titles = tf.keras.preprocessing.sequence.pad_sequences(titles, maxlen=MAXLEN_TITLE, dtype='float32', padding='post', truncating='post')
ledes = tf.keras.preprocessing.sequence.pad_sequences(ledes, maxlen=MAXLEN_LEDE, dtype='float32', padding='post', truncating='post')
contentss = tf.keras.preprocessing.sequence.pad_sequences(contentss, maxlen=MAXLEN_CONTENTS, dtype='float32', padding='post', truncating='post')

np.save(training_data_dir / 'news-{}-{}{}-title.npy'.format(DATA_TYPE, EMBED_NAME, DIM), titles)
np.save(training_data_dir / 'news-{}-{}{}-lede.npy'.format(DATA_TYPE, EMBED_NAME, DIM), ledes)
np.save(training_data_dir / 'news-{}-{}{}-contents.npy'.format(DATA_TYPE, EMBED_NAME, DIM), contentss)

17 1 6.1324727995944315 1.775521857172055
25 3 13.04548999727021 2.2777913642670953
2245 1 155.44556019186524 123.34196657583232
